## Importation des packages

In [1]:
import pickle
import dill as pickle
import pandas as pd
import spacy
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
import re
import sys      
import urllib
from urllib.request import urlopen
from urllib.error import HTTPError, URLError
from bs4 import BeautifulSoup, Tag, NavigableString, Comment
import pickle
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer # ou CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
#from nltk.corpus import stopwords
from google.colab import drive
import tensorflow as tf
import pandas as pd

In [0]:
tf.enable_eager_execution()

In [0]:
#!pip install -U spacy
#!python -m spacy download fr_core_news_sm
nlp = spacy.load('fr_core_news_sm')

## Lecture des fichiers contenant les données, les tokenisers, les matrices de similarité et le modèle bianire

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authentification Google
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# LES DONNEES
id = '1LpPR8X-0uD0_LfBEdfaz-2pooafNGUnX'
downloaded = drive.CreateFile({'id': '1LpPR8X-0uD0_LfBEdfaz-2pooafNGUnX'})
downloaded.GetContentFile('faq')

faq = pickle.load(open('faq', 'rb'))

texts = []
for cl,questions in faq.items() : 
  for question in questions.items():
    texts.append(str(question[0]))
    texts.append(str(question[1]))

# LES MATRICES DE SIMILARITES
id = '1imGqiTPxrml2VOEuSc-O3QXRHDV_Wu0t'
downloaded = drive.CreateFile({'id': '1imGqiTPxrml2VOEuSc-O3QXRHDV_Wu0t'})
downloaded.GetContentFile('matrice_td_tf_idf')
matrice_td_tf_idf = pickle.load(open('matrice_td_tf_idf', 'rb'))

id = '1qWlqs-c8Edu4bI8RtvYZ70GTOYVzvAuo'
downloaded = drive.CreateFile({'id': '1qWlqs-c8Edu4bI8RtvYZ70GTOYVzvAuo'})
downloaded.GetContentFile('matrice_td_bin')
matrice_td_bin = pickle.load(open('matrice_td_bin', 'rb'))

id = '12jelNinkiHHNgdAGhbUHGGI-1wzhPHpq'
downloaded = drive.CreateFile({'id': '12jelNinkiHHNgdAGhbUHGGI-1wzhPHpq'})
downloaded.GetContentFile('matrice_td_n_grams')
matrice_td_n_grams = pickle.load(open('matrice_td_n_grams', 'rb'))

id = '1M_KqN4HD-b4I204V4o2jDLfVnRaNWoHP'
downloaded = drive.CreateFile({'id': '1M_KqN4HD-b4I204V4o2jDLfVnRaNWoHP'})
downloaded.GetContentFile('matrice_td_num')
matrice_td_num = pickle.load(open('matrice_td_num', 'rb'))

id = '1kmtKmutaABSjiAwPTdIBUfP-E0Gg__hK'
downloaded = drive.CreateFile({'id': '1kmtKmutaABSjiAwPTdIBUfP-E0Gg__hK'})
downloaded.GetContentFile('matrice_td_faq')
matrice_td_faq = pickle.load(open('matrice_td_faq', 'rb'))


# LES VECTORISEURS
id = '1GV7Db7h2oaJGF5jb61W0wzwBMbreLd2h'
downloaded = drive.CreateFile({'id': '1GV7Db7h2oaJGF5jb61W0wzwBMbreLd2h'})
downloaded.GetContentFile('vectorizer_faq_bin')
vectorizer_faq_bin = pickle.load(open('vectorizer_faq_bin', 'rb'))

id = '1CbugY_2w5nImbW_NmiHWKLz2BLQDthaV'
downloaded = drive.CreateFile({'id': '1CbugY_2w5nImbW_NmiHWKLz2BLQDthaV'})
downloaded.GetContentFile('vectorizer_faq_num')
vectorizer_faq_num = pickle.load(open('vectorizer_faq_num', 'rb'))

id = '1mz8hBZKANdrB24OkBLgJtwf9QTBzYDgO'
downloaded = drive.CreateFile({'id': '1mz8hBZKANdrB24OkBLgJtwf9QTBzYDgO'})
downloaded.GetContentFile('vectorizer_faq_bin_n_grams')
vectorizer_faq_bin_n_grams = pickle.load(open('vectorizer_faq_bin_n_grams', 'rb'))

id = '1eNezoq7_cmfgJA7oZP13KGsBZ1alRn9g'
downloaded = drive.CreateFile({'id': '1eNezoq7_cmfgJA7oZP13KGsBZ1alRn9g'})
downloaded.GetContentFile('vectorizer_faq_tf_idf')
vectorizer_faq_tf_idf = pickle.load(open('vectorizer_faq_tf_idf', 'rb'))

id = '1qbZnRmjGZMp6_jdmbpWgAK7j5BHUmgEL'
downloaded = drive.CreateFile({'id': '1qbZnRmjGZMp6_jdmbpWgAK7j5BHUmgEL'})
downloaded.GetContentFile('vectorizer_faq')
vectorizer_faq = pickle.load(open('vectorizer_faq', 'rb'))

# LE MODELE BINAIRE ET SON VECTORISEUR
id = '1kRF0HN8Cj91UKTDDuSjipzoEpQlUrMGj'
downloaded = drive.CreateFile({'id': '1kRF0HN8Cj91UKTDDuSjipzoEpQlUrMGj'})
downloaded.GetContentFile('classif_binaire_proba')
model_binaire = pickle.load(open('classif_binaire_proba', 'rb'))

id = '14TJXTz6P33D_xSrC9zXRONIuNk0prf5E'
downloaded = drive.CreateFile({'id': '14TJXTz6P33D_xSrC9zXRONIuNk0prf5E'})
downloaded.GetContentFile('bin_vectorizer_proba')
vector_binaire = pickle.load(open('bin_vectorizer_proba', 'rb'))

Fonctions :       
* La première permet de lemmatiser le texte
* La seconde remplace les urls 
* La troisième remplace les noms des agences par un nom générique

In [0]:
def lem(text):
    """ Met à l'infinitif les verbes et met la racine des mots """
    tw_nlp = nlp(text) # spacy
    tokens = []
    for token in tw_nlp:
        tokens.append(token.lemma_)
    return ' '.join(tokens)

def substitute_url(text, url_replacement='URLEXPR'):
  txt = re.sub(r'(.+?)?(http.?://.+?)\s?(.+?)?',r"\1"+url_replacement+' '+r"\3",text,flags=re.IGNORECASE)
  for i in range(4) : 
    txt = re.sub(r'(.+?)?(http.?://.+?)\s?(.+?)?',r"\1"+url_replacement+' '+r"\3",txt,flags=re.IGNORECASE)
  return txt

def substitute_agence(text, replacement='AGENCE'):
  txt = re.sub(r'(.+?)?(tui.fr)\s?(.+?)?',r"\1"+replacement+' '+r"\3",text,flags=re.IGNORECASE)
  for i in range(4) : 
    txt = re.sub(r'(.+?)?(tui.fr)\s?(.+?)?',r"\1"+replacement+' '+r"\3",txt,flags=re.IGNORECASE)
    txt = re.sub(r'(.+?)?(tui)\s?(.+?)?',r"\1"+replacement+' '+r"\3",txt,flags=re.IGNORECASE)
    txt = re.sub(r'(.+?)?(exotismes)\s?(.+?)?',r"\1"+replacement+' '+r"\3",txt,flags=re.IGNORECASE)
  return txt 

# Evaluation quantitative

Importation du jeu de données test.

In [6]:
# Authentification Google
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
 
id = '1wCRYVRVDJMgsHIhbpw0ndacMwaG0QMUM'
downloaded = drive.CreateFile({'id': '1wCRYVRVDJMgsHIhbpw0ndacMwaG0QMUM'})
downloaded.GetContentFile('Jeu_test')

jeu_test = pd.read_excel('Jeu_test')
print(jeu_test)

                                        Question_test                                  Reponse_souhaitee
0   "La location ne répondait absolument pas à mes...  Si vous souhaitez formuler une réclamation sur...
1           "Je ne suis pas contente de mes vacances"  'Si vous souhaitez formuler une réclamation su...
2   "Mes vacances se sont mal passées, je veux me ...  Si vous souhaitez formuler une réclamation et ...
3   "J'ai loupé l'avion, puis-je me faire rembours...  "Lorsque vous ne prenez pas votre vol, vous av...
4    "Quels sont les différents circuits proposées ?"  "Rendez-vous sur la page circuits https://www....
..                                                ...                                                ...
60  Quels sont les documents à avoir sur moi lors ...  "Préalablement à la conclusion du contrat, le ...
61  Existe-t-il des activités pour les adolescents ?   "Certains des hôtels que nous proposons dispos...
62  Est-il obligé de souscrire à une assurance voy...  

In [0]:
jeu_test.columns = ['Questions', 'Reponses']

In [0]:
jeu_test['Questions'] = [phrase.strip('"').strip("'") for phrase in jeu_test['Questions']] #.apply(strip)

In [0]:
jeu_test['Reponses'] = [phrase.strip('"').strip("'") for phrase in jeu_test['Reponses']] #.apply(strip)

In [0]:
def eval_quanti(ph_entree) : 
  """Prend une phrase en entrée et renvoie ce que le robot renvoie"""
  q = pd.DataFrame([ph_entree])
  q = q[0].apply(lem)
  q_bin = vector_binaire.transform(q)
  q_sim = vectorizer_faq_bin.transform(q)

  # si le modèle prédit 1 (c-a-d que la question de l'utilisateur rentre dans la thématique métier)
  if (model_binaire.predict_proba(q_bin)[0][0]<0.75):
    query_corpus_sim = np.squeeze(cosine_similarity(matrice_td_bin, q_sim))
    idx_most_sim = np.argmax(query_corpus_sim)
    ok = False
    for cl, questions in faq.items():
      for question in questions.items():
        # si la question de l'utilisateur similaire à une question 
        if str(question[0]) == texts[idx_most_sim] : # and 
          # alors on affiche la réponse à cette question
          return question[1]
    # sinon on affiche la réponse qui est la plus similaire
    if ok == False :
      return texts[idx_most_sim]
  # si le classifieur binaire prédit que la question de l'utilisateur n'est pas dans la thématique métier, alors on génère du texte 
  else : 
    return None

In [0]:
#texts

Calcul du score sur le jeu de test.

In [12]:
def scores(ph_entrees, ph_sorties) : 
  """ Calcul et renvoie l'accuracy suivant un jeu de données test """
  col_res = []
  for idx in range(len(ph_entrees)) : 
    # print("#####################")
    # print("Q :", ph_entrees[idx])
    # print("RV :", ph_sorties[idx])
    # print("RR : ", eval_quanti(ph_entrees[idx]))
    if eval_quanti(ph_entrees[idx]) ==  ph_sorties[idx] : # renvoie la bonne phrase
      col_res.append(1)
    else : # ne renvoie pas la bonne phrase
      col_res.append(0)
  
  accuracy = sum(col_res)/len(col_res)
  
  return accuracy

scores(jeu_test['Questions'], jeu_test['Reponses'])

0.2923076923076923

In [13]:
def eval_quanti(ph_entree) : 
  """Prend une phrase en entrée et renvoie ce que le robot renvoie"""
  q = pd.DataFrame([ph_entree])
  q = q[0].apply(lem)
  q_bin = vector_binaire.transform(q)
  q_sim = vectorizer_faq_bin_n_grams.transform(q)

  # si le modèle prédit 1 (c-a-d que la question de l'utilisateur rentre dans la thématique métier)
  if (model_binaire.predict_proba(q_bin)[0][0]<0.75):
    query_corpus_sim = np.squeeze(cosine_similarity(matrice_td_n_grams, q_sim))
    idx_most_sim = np.argmax(query_corpus_sim)
    ok = False
    for cl, questions in faq.items():
      for question in questions.items():
        # si la question de l'utilisateur similaire à une question 
        if str(question[0]) == texts[idx_most_sim] : # and 
          # alors on affiche la réponse à cette question
          return question[1]
    # sinon on affiche la réponse qui est la plus similaire
    if ok == False :
      return texts[idx_most_sim]
  # si le classifieur binaire prédit que la question de l'utilisateur n'est pas dans la thématique métier, alors on génère du texte 
  else : 
    return None
scores(jeu_test['Questions'], jeu_test['Reponses'])

0.18461538461538463

In [14]:
def eval_quanti(ph_entree) : 
  """Prend une phrase en entrée et renvoie ce que le robot renvoie"""
  q = pd.DataFrame([ph_entree])
  q = q[0].apply(lem)
  q_bin = vector_binaire.transform(q)
  q_sim = vectorizer_faq_num.transform(q)

  # si le modèle prédit 1 (c-a-d que la question de l'utilisateur rentre dans la thématique métier)
  if (model_binaire.predict_proba(q_bin)[0][0]<0.75):
    query_corpus_sim = np.squeeze(cosine_similarity(matrice_td_num, q_sim))
    idx_most_sim = np.argmax(query_corpus_sim)
    ok = False
    for cl, questions in faq.items():
      for question in questions.items():
        # si la question de l'utilisateur similaire à une question 
        if str(question[0]) == texts[idx_most_sim] : # and 
          # alors on affiche la réponse à cette question
          return question[1]
    # sinon on affiche la réponse qui est la plus similaire
    if ok == False :
      return texts[idx_most_sim]
  # si le classifieur binaire prédit que la question de l'utilisateur n'est pas dans la thématique métier, alors on génère du texte 
  else : 
    return None
scores(jeu_test['Questions'], jeu_test['Reponses'])

0.26153846153846155

In [15]:
def eval_quanti(ph_entree) : 
  """Prend une phrase en entrée et renvoie ce que le robot renvoie"""
  q = pd.DataFrame([ph_entree])
  q = q[0].apply(lem)
  q_bin = vector_binaire.transform(q)
  q_sim = vectorizer_faq_tf_idf.transform(q)

  # si le modèle prédit 1 (c-a-d que la question de l'utilisateur rentre dans la thématique métier)
  if (model_binaire.predict_proba(q_bin)[0][0]<0.75):
    query_corpus_sim = np.squeeze(cosine_similarity(matrice_td_tf_idf, q_sim))
    idx_most_sim = np.argmax(query_corpus_sim)
    ok = False
    for cl, questions in faq.items():
      for question in questions.items():
        # si la question de l'utilisateur similaire à une question 
        if str(question[0]) == texts[idx_most_sim] : # and 
          # alors on affiche la réponse à cette question
          return question[1]
    # sinon on affiche la réponse qui est la plus similaire
    if ok == False :
      return texts[idx_most_sim]
  # si le classifieur binaire prédit que la question de l'utilisateur n'est pas dans la thématique métier, alors on génère du texte 
  else : 
    return None
scores(jeu_test['Questions'], jeu_test['Reponses'])

0.2153846153846154

In [16]:
def eval_quanti(ph_entree) : 
  """Prend une phrase en entrée et renvoie ce que le robot renvoie"""
  q = pd.DataFrame([ph_entree])
  q = q[0].apply(lem).apply(substitute_url).apply(substitute_agence)
  q_bin = vector_binaire.transform(q)
  q_sim = vectorizer_faq.transform(q)

  # si le modèle prédit 1 (c-a-d que la question de l'utilisateur rentre dans la thématique métier)
  if (model_binaire.predict_proba(q_bin)[0][0]<0.85):
    query_corpus_sim = np.squeeze(cosine_similarity(matrice_td_faq, q_sim))
    idx_most_sim = np.argmax(query_corpus_sim)
    ok = False
    for cl, questions in faq.items():
      for question in questions.items():
        # si la question de l'utilisateur similaire à une question 
        if str(question[0]) == texts[idx_most_sim] : # and 
          # alors on affiche la réponse à cette question
          return question[1]
    # sinon on affiche la réponse qui est la plus similaire
    if ok == False :
      return texts[idx_most_sim]
  # si le classifieur binaire prédit que la question de l'utilisateur n'est pas dans la thématique métier, alors on génère du texte 
  else : 
    return None
scores(jeu_test['Questions'], jeu_test['Reponses'])

0.3230769230769231

On garde la matrice terme-documents binaire avec les n-grams et les fonctions de pré-traitement supplémentaires, car son score est le plus grand (0.32). On note aussi qu'on augmente la proba de la classe thématique à 0.85.